In [1]:
# Importando bibliotecas
import time
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen

import bs4

#https://www.camaraindaial.sc.gov.br/pg/proposicoes
#https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=806&aaProposicao=2021

In [2]:
# carregando variável com valor do "agente"
agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': agente}

In [5]:
def consulta_web(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [6]:
def capture_html_page(url):
    html = consulta_web(url)
    
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

In [8]:
# está pegando apenas o dt e o dd (tabela que já está na pagina)
def get_data(html):
    
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic[x] = y
        
    if not bool(dic):
        raise Exception('Proposição inexistente')
        
    return dic

In [13]:
# adiciona o restante das informações da proposicao ao dicionário 
def get_proposicao_data(proposicao, ano):
    url = f'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao={str(proposicao)}&aaProposicao={str(ano)}'
    html = capture_html_page(url)
    dic = get_data(html)
    
    dic['Proposicao'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text() # forma direta do b4s
        
    return dic

In [16]:
def get_content(beginning, quantity, year, acceptable_errors, wait_time):
    last_query = beginning + quantity - 1
    
    # errors
    errors = 0
    
    # loop vars
    i = 1
    list = []
    
    while beginning <= last_query and errors <= acceptable_errors:
        try:
            proposicao = get_proposicao_data(beginning, year)
            list += [proposicao]
        except:
            errors += 1
            pass
        
        time.sleep(wait_time)
        
        beginning += 1
        i+=1
    
    return pd.DataFrame(list)

# Inserindo dados no SQL Server

In [17]:
# function only for tests
teste = get_content(401, 4, 2021, 2, 0.2)
teste

,Reunião,Deliberação,Situação,Assunto,Autor,Proposicao,Ano,Texto
0,15/03/2021,15/03/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Roger Michel Knipers.,401,2021,"O vereador abaixo firmado requer, após ouvido ..."
1,15/03/2021,15/03/2021,Proposição Aprovada,Manutenção de via pública,Vereador Ana Paula Reiter.,402,2021,"A vereadora abaixo firmada requer, após ouvido..."
2,18/03/2021,NaN,Entrada na Ordem do Dia,Manutenção de via pública,Vereador Jonas Luiz de Lima.,403,2021,"O Vereador abaixo firmado requer, após ouvido ..."
3,18/03/2021,NaN,Entrada na Ordem do Dia,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,404,2021,"O vereador que está subscreve, no uso das atri..."


```sql
CREATE TABLE Proposicoes (
    DataReuniao DATE,
    DataDeliberacao DATE,
    Situacao VARCHAR(200),
    Assunto VARCHAR(1000),
    Autor VARCHAR(1000),
    Proposicao INT,
    Ano INT,
    Texto VARCHAR(MAX)
)
```

In [14]:
import pyodbc

In [16]:
# Conexão no banco, tirando UID e PWD vira Windows Authentication
conn = pyodbc.connect('Trusted_Connection=yes',
                     driver = '{ODBC Driver 17 for SQL Server}',
                     server = 'localhost',
                     database = 'Indaial',
                     UID='sa',
                     PWD='123')

query = '''
Select * from proposicoes
'''

sql_query = pd.read_sql_query(query, conn)
sql_query

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


> 55.43 Algumas consultas no site não vem com a coluna Deliberação, vamos arrumar esse problema:
Ex: get_content(403, 4, 2021, 2, 0.2)

In [45]:
#exemplo
teste = get_content(403, 4, 2021, 2, 0.2)
teste

,Reunião,Situação,Assunto,Autor,Proposicao,Ano,Texto
0,18/03/2021,Entrada na Ordem do Dia,Manutenção de via pública,Vereador Jonas Luiz de Lima.,403,2021,"O Vereador abaixo firmado requer, após ouvido ..."
1,18/03/2021,Entrada na Ordem do Dia,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,404,2021,"O vereador que está subscreve, no uso das atri..."
2,18/03/2021,Entrada na Ordem do Dia,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,405,2021,"O vereador que está subscreve, no uso das atri..."
3,18/03/2021,Entrada na Ordem do Dia,Manutenção de via pública,Vereador Flávio Augusto Ferri Molinari.,406,2021,"O vereador que está subscreve, no uso das atri..."


In [60]:
# DataFrame base para evitar coluna faltante
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposicao', 'Ano', 'Texto'])

# Junção dos dados com a base e tratamento de NaN
teste = base.append(teste).fillna('01/01/1990')

# Transformação para tipo date
teste['Reunião'] = pd.to_datetime(teste['Reunião'])
teste['Deliberação'] = pd.to_datetime(teste['Deliberação'])

conn = pyodbc.connect('Trusted_Connection=yes',
                     driver = '{ODBC Driver 17 for SQL Server}',
                     server = 'localhost',
                     database = 'Indaial')

cursor = conn.cursor()

for index, row in teste.iterrows():
    cursor.execute('''
        INSERT INTO Proposicoes (
            DataReuniao,
            DataDeliberacao,
            Situacao,
            Assunto,
            Autor,
            Proposicao,
            Ano,
            Texto
        )
        values(?,?,?,?,?,?,?,?)''',
        row['Reunião'],
        row['Deliberação'],
        row['Situação'],
        row['Assunto'],
        row['Autor'],
        row['Proposicao'],
        row['Ano'],
        row['Texto']
    )

conn.commit()
cursor.close()

In [15]:
def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes',
                     driver = '{ODBC Driver 17 for SQL Server}',
                     server = 'localhost',
                     database = 'Indaial')
    
    out = pd.read_sql_query(query, conn)
    return out

In [16]:
SQLSelect('select * from proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-01-02,2021-01-02,Proposição Aprovada,Trânsito,Vereador Remir José de Faveri.,1,2021,"O vereador que esta subscreve, no uso das atri..."
1,2021-01-02,2021-01-02,Proposição Aprovada,Manutenção de via pública,Vereador Remir José de Faveri.,2,2021,"O vereador abaixo firmado requer, após ouvido ..."
2,2021-01-02,2021-01-02,Proposição Aprovada,Manutenção de via pública,Vereador Valentim Blasius.,3,2021,"O vereador abaixo firmado requer, após ouvido ..."


In [54]:
SQLSelect('select maior=max(Proposicao) from Proposicoes')

,maior
0,406


In [33]:
def SQLInsertProposicoes(TabelaProposicoes):
    # DataFrame base para evitar coluna faltante
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposicao', 'Ano', 'Texto'])

    # Junção dos dados com a base e tratamento de NaN
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('01/01/1990')

    # Transformação para tipo date
    TabelaProposicoes['Reunião'] = pd.to_datetime(TabelaProposicoes['Reunião'])
    TabelaProposicoes['Deliberação'] = pd.to_datetime(TabelaProposicoes['Deliberação'])

    conn = pyodbc.connect('Trusted_Connection=yes',
                         driver = '{ODBC Driver 17 for SQL Server}',
                         server = 'localhost',
                         database = 'Indaial')

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():
        cursor.execute('''
            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            )
            values(?,?,?,?,?,?,?,?)''',
            row['Reunião'],
            row['Deliberação'],
            row['Situação'],
            row['Assunto'],
            row['Autor'],
            row['Proposicao'],
            row['Ano'],
            row['Texto']
        )

    conn.commit()
    cursor.close()

In [18]:
def SQLTruncate(NomeTabela):
    conn = pyodbc.connect('Trusted_Connection=yes',
                         driver = '{ODBC Driver 17 for SQL Server}',
                         server = 'localhost',
                         database = 'Indaial')

    cursor = conn.cursor()
    
    cursor.execute(f'TRUNCATE TABLE {NomeTabela}')
    
    conn.commit()
    cursor.close()

In [80]:
SQLTruncate('Proposicoes')

In [88]:
# ==== INICIO ====
ano = 2021
dados_ano = SQLSelect(f'select proposicao = max(proposicao) from proposicoes where ano = {ano}')
ultima_proposicao = dados_ano['proposicao'].loc[0]

# define o número da proxima proposicao
if ultima_proposicao == None:
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao) + 1


dados = get_proposicao_data(proxima_proposicao, ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)

SQLSelect('select * from proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-01-02,2021-01-02,Proposição Aprovada,Trânsito,Vereador Remir José de Faveri.,1,2021,"O vereador que esta subscreve, no uso das atri..."
1,2021-01-02,2021-01-02,Proposição Aprovada,Manutenção de via pública,Vereador Remir José de Faveri.,2,2021,"O vereador abaixo firmado requer, após ouvido ..."


In [34]:
def InsereProximaProposicao(ano):
    # pega a última proposição do ano
    dados_ano = SQLSelect(f'select proposicao = max(proposicao) from proposicoes where ano = {ano}')
    
    
    ultima_proposicao = dados_ano['proposicao'].loc[0]
    
    # define o número da proxima proposicao
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1
        
    dados = get_proposicao_data(proxima_proposicao, ano)
    tabela = pd.DataFrame([dados])
    
    SQLInsertProposicoes(tabela)

In [36]:
InsereProximaProposicao(2023)
SQLSelect('select * from proposicoes')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2021-01-02,2021-01-02,Proposição Aprovada,Trânsito,Vereador Remir José de Faveri.,1,2021,"O vereador que esta subscreve, no uso das atri..."
1,2021-01-02,2021-01-02,Proposição Aprovada,Manutenção de via pública,Vereador Remir José de Faveri.,2,2021,"O vereador abaixo firmado requer, após ouvido ..."
2,2021-01-02,2021-01-02,Proposição Aprovada,Manutenção de via pública,Vereador Valentim Blasius.,3,2021,"O vereador abaixo firmado requer, após ouvido ..."
3,2021-01-02,2021-01-02,Proposição Aprovada,Manutenção de via pública,Vereador Valentim Blasius.,4,2021,"O vereador abaixo firmado requer, após ouvido ..."
4,2023-02-02,2023-02-02,Proposição Despachada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Remir José de Faveri.,1,2023,"O vereador que esta subscreve, no uso das atri..."


In [61]:
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 5, segundos_espera = 0):
    erros = 0

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

    time.sleep(segundos_espera)

In [60]:
SQLTruncate('proposicoes')

In [64]:
ano_inicio = 1996
ano_fim = 1998

for ano in list(range(ano_inicio, ano_fim+1)):
    print(f"iniciando inserção dos dados do ano de {ano}")
    try:
        BuscaGravaDadosAno(ano)
    except:
        pass

    print('Inserção finalizada com sucesso!')

iniciando inserção dos dados do ano de 1996
Inserção finalizada com sucesso!
iniciando inserção dos dados do ano de 1997
Inserção finalizada com sucesso!
iniciando inserção dos dados do ano de 1998
Inserção finalizada com sucesso!


In [42]:
x = range(1996, 1998)

range(1996, 1998)

> Houve uma alteração no site provavelmente... No vídeo o Joviano diz que 1996 só possui 24 proposições, mas na verdade vai até a 399. A 25 por exemplo não existem, porém a pagina é retornada sem erro, o dicionário fica nulo e o texto obtido é o texto de solicitação de cookies. Para contornar isso, na função "get_data" eu coloquei uma Exception para dar erro quando o dicionário vier vazio, mas não está pegando todas as de 1996